In [1]:
from utils2 import MWU, ttests, qvalues
import pandas as pd
import numpy as np
import time
from significance_of_mean_cuda import significance_of_mean_cuda

In [2]:
%load_ext cython
%load_ext rpy2.ipython

R[write to console]: Warning:
R[write to console]:  namespace ‘pbdZMQ’ is not available and has been replaced
by .GlobalEnv when processing object ‘.pbd_env’

/home/ekvall/anaconda3/envs/my-r-env/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/home/ekvall/anaconda3/envs/my-r-env/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [3]:
%%R

require(ggplot2)
install.packages("devtools")
library("devtools")
install_github("bdsegal/fastPerm")

library(fastPerm)

R[write to console]: Loading required package: ggplot2

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/devtools_2.3.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 375227 bytes (366 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to cons

In [4]:
url = "https://static-content.springer.com/esm/art%3A10.1038%2Fnature18003/MediaObjects/41586_2016_BFnature18003_MOESM111_ESM.zip"
!mkdir "data/"
!wget -P "data/" "https://static-content.springer.com/esm/art%3A10.1038%2Fnature18003/MediaObjects/41586_2016_BFnature18003_MOESM111_ESM.zip"
!unzip "./data/41586_2016_BFnature18003_MOESM111_ESM.zip" -d "./data/"

--2020-04-15 18:00:24--  https://static-content.springer.com/esm/art%3A10.1038%2Fnature18003/MediaObjects/41586_2016_BFnature18003_MOESM111_ESM.zip
Resolving static-content.springer.com (static-content.springer.com)... 151.101.84.95
Connecting to static-content.springer.com (static-content.springer.com)|151.101.84.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138716400 (132M) [application/zip]
Saving to: ‘data/41586_2016_BFnature18003_MOESM111_ESM.zip’

41586_2016_BFnature 100%[===================>] 132.29M  21.5MB/s    in 6.1s    

2020-04-15 18:00:30 (21.8 MB/s) - ‘data/41586_2016_BFnature18003_MOESM111_ESM.zip’ saved [138716400/138716400]

Archive:  ./data/41586_2016_BFnature18003_MOESM111_ESM.zip
   creating: ./data/nature18003-s2/
  inflating: ./data/nature18003-s2/CPTAC_BC_SupplementaryTable01.xlsx  
   creating: ./data/__MACOSX/
   creating: ./data/__MACOSX/nature18003-s2/
  inflating: ./data/__MACOSX/nature18003-s2/._CPTAC_BC_SupplementaryTable01.

In [5]:
path = "./data/nature18003-s2/"
T1 = pd.read_excel(path + "CPTAC_BC_SupplementaryTable01.xlsx")
T3 = pd.read_excel(path + "CPTAC_BC_SupplementaryTable03.xlsx")

## Extract important column headings

In [6]:
paiteint_id = T1.columns[0]
stats_col = T1.columns[5: 5 + 3]

### Patient column  

In [7]:
paiteint_id

'Sample ID'

### Cancer phenotype coulmns

In [8]:
stats_col

Index(['ER Status', 'PR Status', 'HER2 Status'], dtype='object')

### Check viable phenotype status

In [9]:
all_lab = list()
for r in T1[stats_col].iterrows():
    all_lab += list(r[1].values)

In [10]:
list(set(all_lab))

['Positive', 'Negative', 'Indeterminate', 'Equivocal']

### Let's drop patient with "Equivocal" and "Indeterminate" status

In [11]:
drop_label = ['Indeterminate', 'Equivocal']

In [12]:
T1.head()

,Sample ID,TCGA ID,Biospecimen Barcode Side,UIUD,PAM50,ER Status,PR Status,HER2 Status,QC Status,Proteome Cluster (see Fig. 3b),...,iTRAQReporterIon,CommonControl,GATA3 Mutation,PIK3CA Mutation,TP53 Mutation,PIK3CA missense mutation in helical domain all tumors,PIK3CA missense mutation in kinase domain all tumors,TP53 Nonsense/Frameshift all tumors,TP53 Missense mutation all tumors,TP53 missense mutations in DNA binding domain all tumors
0,A2-A0CM,TCGA-A2-A0CM,TCGA-A2-A0CM-01A-11-A21V-30,330F7598-824C-4CD6-9303-A27FE74A6695,Basal,Negative,Negative,Negative,pass,1.0,...,116,YES,NaN,NaN,Frame_Shift_Del|Somatic|p.E204fs,0.0,0.0,1.0,NaN,NaN
1,A2-A0D2,TCGA-A2-A0D2,TCGA-A2-A0D2-01A-12-A21W-30,308CCD79-C164-4397-92BC-A1CD243C8E7D,Basal,Negative,Negative,Negative,pass,1.0,...,114,NO,NaN,NaN,Frame_Shift_Del|Somatic|p.P318fs,0.0,0.0,1.0,NaN,NaN
2,A2-A0EQ,TCGA-A2-A0EQ,TCGA-A2-A0EQ-01A-41-A21W-30,04217F65-2907-478B-B0C5-EB65370198DA,Her2,Negative,Negative,Positive,pass,1.0,...,116,NO,NaN,Missense_Mutation|Somatic|p.H1047R,In_Frame_Del|Somatic|p.IY162in_frame_delN,NaN,1.0,NaN,NaN,NaN
3,A2-A0EV,TCGA-A2-A0EV,TCGA-A2-A0EV-01A-41-A21V-30,38F98E5F-7FA4-4C89-8D81-516CF865BEEB,LumA,Positive,Positive,Negative,pass,3.0,...,114,NO,NaN,In_Frame_Ins|Somatic|p.E469in_frame_insDK,NaN,NaN,NaN,0.0,0.0,0.0
4,A2-A0EX,TCGA-A2-A0EX,TCGA-A2-A0EX-01A-41-A21V-30,78271500-147B-40C5-B789-0CF0C7CDBCE2,LumA,Positive,Positive,Negative,pass,3.0,...,116,NO,NaN,Missense_Mutation|Somatic|p.E545K,NaN,1.0,NaN,0.0,0.0,0.0


In [13]:
mask1 = [True if (len(set(r[1].values) & set(drop_label)) == 0) else False for r in T1[stats_col].iterrows()]
T1 = T1[mask1]
T1.reset_index(drop=True, inplace=True)

### Let's divide the dataframe into two dataframes with triple negative (TN) and non-triple negative (NTN)

In [14]:
mask2 = [all([True if v=='Negative' else False for v in r[1].values] ) for r in T1[stats_col].iterrows()]

T1_TNP = T1[np.array(mask2)]
T1_TNP.reset_index(drop=True, inplace=True)

T1_not_TNP = T1[~np.array(mask2)]
T1_not_TNP.reset_index(drop=True, inplace=True)



### Obtain patient ids for NT and NTN

In [15]:
TNPpateintIds = T1_TNP[paiteint_id].values
NotTNPpateintIds = T1_not_TNP[paiteint_id].values

### Remove genes (rows) containing NaN for both TN and NTN i.e., make sure all patients have same the gene when comparing.

In [16]:
def getpatientId(ids):
    patitentList = list()
    for i in ids:
        p_id = T3[T3.columns[[i in c for c in T3.columns]]].columns
        patitentList += list(p_id)
    return patitentList

In [17]:
only_patientDf = T3[getpatientId(NotTNPpateintIds) + getpatientId(TNPpateintIds)]

In [18]:
only_patientDf.dropna(inplace=True)

/home/ekvall/anaconda3/envs/my-r-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
NotTNP_df =  only_patientDf[getpatientId(NotTNPpateintIds)]
TNP_df =  only_patientDf[getpatientId(TNPpateintIds)]

In [20]:
NotTNP_Arr, TNP_Arr =NotTNP_df.values, TNP_df.values

In [21]:
NotTNP_Arr.shape

(8051, 80)

In [22]:
TNP_Arr.shape

(8051, 26)

In [23]:
#NotTNP_df.to_csv("data/notTNPdf")
#TNP_df.to_csv("data/TNPdf")

### There are 80 patients without triple negative (NTN) and 26 patients with triple negtive (TN), and with a total of 8051 quantified genes.

# Experiment time

## Let's start with a rather small binsize=6.

In [24]:
start = time.time()

SGM = significance_of_mean_cuda(6, dtype_v=np.uint32,dtype_A=np.float64)
SGM.run(TNP_Arr, NotTNP_Arr)
P_e_list = [2 * min( p, (1-p)) for p in SGM.get_p_values()]

end = time.time()
print(end - start)

This data requires 1123.404336 MiB on the GPU.
2.2298593521118164


In [25]:
x = pd.DataFrame(TNP_Arr, columns=list(range(TNP_Arr.shape[1]))).T
y = pd.DataFrame(NotTNP_Arr, columns=list(range(NotTNP_Arr.shape[1]))).T
n_samples = x.shape[1]
outputDf = pd.DataFrame(n_samples * [0], columns=["output"]).T

In [26]:
%R -i x
%R -i y
%R -i outputDf
%R -i n_samples

In [27]:
%%R
runFastPerm <- function(x,y,output, n_samples) {
    e <- list(mode="vector",length=n_samples)
        for (i in 1:n_samples) {
      
            mStopDiffMean(x[[i]], y[[i]])
            valX = fastPerm(x[[i]], y[[i]], testStat = diffMean)
            
            print(valX)
            valX <-unlist(valX)

      
            output[[i]] <- as.numeric(valX[1])
      
            }
    

    return(output)


}


In [28]:
start = time.time()
%R out <- runFastPerm(x,y, outputDf, n_samples)
end = time.time()
print(end - start)

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0769
p-value = 0.797
mStop = 20, deviance = 190, AIC = 372
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0734
p-value = 0.804
mStop = 20, deviance = 182, AIC = 364
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0706
p-value = 0.815
mStop = 20, deviance = 175, AIC = 357
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.076
p-value = 0.796
mStop = 20, deviance = 221, AIC = 403
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0742
p-value = 0.803
mStop = 20, deviance = 210, AIC = 392
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.073
p-value = 0.799
mStop = 20, de

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.801
p-value = 0.0182
mStop = 20, deviance = 364, AIC = 503
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.74
p-value = 0.000591
mStop = 18, deviance = 306, AIC = 404
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.731
p-value = 0.000478
mStop = 19, deviance = 287, AIC = 387
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.724
p-value = 0.000977
mStop = 19, deviance = 329, AIC = 432
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.307
p-value = 0.272
mStop = 20, deviance = 509, AIC = 675
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0465
p-value = 0.929
mStop = 

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.356
p-value = 0.148
mStop = 20, deviance = 523, AIC = 682
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.65
p-value = 0.000327
mStop = 16, deviance = 329, AIC = 419
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.71
p-value = 5.4e-05
mStop = 13, deviance = 257, AIC = 333
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.76
p-value = 1.05e-05
mStop = 12, deviance = 246, AIC = 315
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.762
p-value = 0.0341
mStop = 20, deviance = 395, AIC = 539
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.335
p-value = 0.325
mStop = 20, 

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.474
p-value = 0.211
mStop = 20, deviance = 577, AIC = 740
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.483
p-value = 0.206
mStop = 20, deviance = 588, AIC = 751
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.503
p-value = 0.137
mStop = 20, deviance = 502, AIC = 661
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.267
p-value = 0.0362
mStop = 20, deviance = 405, AIC = 551
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.5
p-value = 7.09e-05
mStop = 15, deviance = 216, AIC = 297
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.979
p-value = 0.0176
mStop = 20, dev

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.09
p-value = 0.000227
mStop = 17, deviance = 306, AIC = 399
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.016
p-value = 0.971
mStop = 20, deviance = 9.08, AIC = 196
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.811
p-value = 0.000464
mStop = 17, deviance = 317, AIC = 411
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.499
p-value = 0.00241
mStop = 20, deviance = 324, AIC = 442
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.501
p-value = 0.000624
mStop = 16, deviance = 302, AIC = 396
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0414
p-value = 0.912
mStop 

...

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.451
p-value = 0.132
mStop = 20, deviance = 548, AIC = 706
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0603
p-value = 0.786
mStop = 20, deviance = 205, AIC = 386
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.181
p-value = 0.327
mStop = 20, deviance = 516, AIC = 684
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.123
p-value = 0.6
mStop = 20, deviance = 337, AIC = 514
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.439
p-value = 0.262
mStop = 20, deviance = 485, AIC = 651
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.113
p-value = 0.826
mStop = 20, devianc

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.743
p-value = 0.00506
mStop = 20, deviance = 375, AIC = 500
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0642
p-value = 0.809
mStop = 20, deviance = 160, AIC = 342
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.249
p-value = 0.0288
mStop = 20, deviance = 382, AIC = 525
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.586
p-value = 0.38
mStop = 20, deviance = 476, AIC = 646
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.469
p-value = 0.269
mStop = 20, deviance = 546, AIC = 712
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.369
p-value = 0.281
mStop = 20, dev

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.309
p-value = 0.672
mStop = 20, deviance = 274, AIC = 452
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.429
p-value = 0.268
mStop = 20, deviance = 588, AIC = 753
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.944
p-value = 0.0229
mStop = 20, deviance = 401, AIC = 542
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.572
p-value = 0.367
mStop = 20, deviance = 453, AIC = 622
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.344
p-value = 0.222
mStop = 20, deviance = 522, AIC = 685
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0763
p-value = 0.891
mStop = 20, devi

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0309
p-value = 0.969
mStop = 20, deviance = 15.9, AIC = 202
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0977
p-value = 0.673
mStop = 20, deviance = 309, AIC = 487
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0682
p-value = 0.644
mStop = 20, deviance = 356, AIC = 533
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.261
p-value = 0.595
mStop = 20, deviance = 322, AIC = 499
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.134
p-value = 0.472
mStop = 20, deviance = 450, AIC = 622
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 1.3
p-value = 0.0223
mStop = 20, dev

    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.1
p-value = 0.789
mStop = 20, deviance = 215, AIC = 396
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.45
p-value = 0.686
mStop = 20, deviance = 284, AIC = 462
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.439
p-value = 0.0451
mStop = 20, deviance = 400, AIC = 548
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.108
p-value = 0.714
mStop = 20, deviance = 291, AIC = 470
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.0147
p-value = 0.981
mStop = 20, deviance = 3.91, AIC = 191
    fastPerm Two Sample Test   

difference of means
1,000 iterations within partitions

observed statistic = 0.132
p-value = 0.763
mStop = 20, devian

In [30]:
2738 / 60

45.63333333333333

In [31]:
%R -o out

In [32]:
outT = out.T

In [33]:
P_fp_list =list(outT.values.ravel())

### Parallelized exact test takes ~3s

In [34]:
start = time.time()

P_mw_list = MWU(TNP_Arr, NotTNP_Arr)

end = time.time()
print(end - start)

1.5351297855377197


### Mann-Whitney exact test takes ~1s

In [35]:
start = time.time()

P_t_list = ttests(TNP_Arr, NotTNP_Arr, True)

end = time.time()
print(end - start)

1.4076178073883057


In [36]:
df_e = pd.DataFrame({"p":P_e_list})
df_mw = pd.DataFrame({"p":P_mw_list})
df_tt = pd.DataFrame({"p":P_t_list})
df_fp = pd.DataFrame({"p":list(np.array(P_fp_list) )})

In [37]:
df_e = qvalues(df_e, pi0=None)
df_mw = qvalues(df_mw, pi0=None)
df_tt = qvalues(df_tt, pi0=None)
df_fp = qvalues(df_fp, pi0=None)

In [38]:
df_e["q"][df_e["q"]<0.01].shape[0]


774

In [39]:
df_mw["q"][df_mw["q"]<0.01].shape[0]

974

In [40]:
df_tt["q"][df_tt["q"]<0.01].shape[0]

351

In [41]:
df_fp["q"][df_fp["q"]<0.01].shape[0]

1005